In [5]:
######################################################################
# LIBRERÍAS
######################################################################
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import matplotlib.pyplot as plt
import os
import timeit
mytimer=lambda:timeit.default_timer()

In [6]:
######################################################################
# CREACIÓN DEL DIRECTORIO DE TRABAJO
######################################################################
local_dir = "../"
dirdata = os.getcwd() + "/" + local_dir

In [7]:
######################################################################
# LECTURA DE ARCHIVOS
######################################################################
# Description:
#
# hip : Hipparcos identifier (int)
# tycho2_id : Tycho 2 identifier (string)
# ref_epoch : Reference epoch (double, Time[Julian Years]), expressed as a Julian Year in TCB.
# ra : Right ascension (double, Angle[deg]). Barycentric right ascension of the source in ICRS at the reference epoch ref_epoch.
# ra error : Standard error of right ascension (double, Angle[mas])
# dec : Declination (double, Angle[deg]). Barycentric declination of the source in ICRS at the reference epoch ref_epoch.
# dec error : Standard error of declination (double, Angle[mas])
# parallax : Parallax (double, Angle[mas]). Absolute barycentric stellar parallax$of the soure at the reference epoch ref_epoch
# parallax error : Standard error of parallax (double, Angle[mas] )
# pmra : Proper motion in right ascension direction (double, Angular Velocity[mas/year]
# pmra error : Standard error of proper motion in right ascension direction (double, Angular Velocity[mas/year])
# pmdec : Proper motion in declination direction (double, Angular Velocity[mas/year]
# pmdec error : Standard error of proper motion in declination direction (double, Angular Velocity[mas/year])
# phot g mean mag : G-band mean magnitude (double, Magnitude[mag]) Mean magnitude in the G band.
# l : Galactic longitude (double, Angle[deg])
# b : Galactic latitude (double, Angle[deg])

cols = ["hip", "tycho2_id", "ref_epoch", "ra", "ra_error", "dec", "dec_error", "parallax", "parallax_error", \
        "pmra", "pmra_error", "pmdec", "pmdec_error", "phot_g_mean_mag", "l", "b"]

for i in range(16):
    file1 = "TgasSource_000-000-0"
    file2 = ".csv.gz"
    file = file1 + str(i).zfill(2)+ file2
    filename = local_dir + "/TGAS/" + file
    if i == 0:
        print("Reading", file)
        DR1 = pd.read_csv(filename) #, usecols=cols)
    else:
        print("Reading", file)
        DRx = pd.read_csv(filename) #, usecols=cols)
        DR1 = DR1.append(DRx)

DR1 = pd.DataFrame(DR1)

Reading TgasSource_000-000-000.csv.gz
Reading TgasSource_000-000-001.csv.gz
Reading TgasSource_000-000-002.csv.gz
Reading TgasSource_000-000-003.csv.gz
Reading TgasSource_000-000-004.csv.gz
Reading TgasSource_000-000-005.csv.gz
Reading TgasSource_000-000-006.csv.gz
Reading TgasSource_000-000-007.csv.gz
Reading TgasSource_000-000-008.csv.gz
Reading TgasSource_000-000-009.csv.gz
Reading TgasSource_000-000-010.csv.gz
Reading TgasSource_000-000-011.csv.gz
Reading TgasSource_000-000-012.csv.gz
Reading TgasSource_000-000-013.csv.gz
Reading TgasSource_000-000-014.csv.gz
Reading TgasSource_000-000-015.csv.gz


In [8]:
######################################################################
# FIX THE DR1 DATABASE
######################################################################
cats=['hip','tycho2_id']
for cname in cats:
    DR1[cname]=DR1[cname].fillna('')
    DR1[cname]=DR1[cname].map(str)
dfstr=DR1.select_dtypes(['object'])
DR1[dfstr.columns]=dfstr.apply(lambda x: x.str.strip())
DR1['hip']=DR1['hip'].apply(lambda x:x.replace('.0',''))

In [9]:
######################################################################
# SELECTED GAIA DATABASE
######################################################################
GAIA=DR1
#GAIA=TGAS

In [10]:
######################################################################
# LEE LA BASE DE DATOS DE RVcat
######################################################################
RV=pd.read_csv(dirdata+"RVGaia/RVCat.csv")
cats=['hip','tycho2_id']
for cname in cats:
    RV[cname]=RV[cname].fillna('')
    RV[cname]=RV[cname].map(str)
dfstr=RV.select_dtypes(['object'])
RV[dfstr.columns]=dfstr.apply(lambda x: x.str.strip())
RV['hip']=RV['hip'].apply(lambda x:x.replace('.0',''))
print("Number of RV objects:",len(RV))

Number of RV objects: 519253


In [ ]:
######################################################################
#POPULATE RVgaia
######################################################################
k=0
RVgaia=pd.DataFrame()
columns=GAIA.columns.tolist()+["RV","eRV"]
Nfreq=10
Ntot=10*Nfreq
ti=mytimer()
for TYC2,HIP in zip(RV.TYC2.values,RV.HIP.values):
    if (k%Nfreq)==0:
        te=mytimer()
        print("Entry %d (time = %.3e)..."%(k,te-ti))
        ti=mytimer()
    row=RV.iloc[k]
    match=pd.DataFrame()
    if TYC2!='':match=GAIA[GAIA.tycho2_id==TYC2]
    if HIP!='':
        if len(match)==0:match=GAIA[GAIA.hip==HIP]
    if len(match)>0:
        found=match.iloc[0]
        nrow=pd.concat([found,row[['RV','eRV']]])
        RVgaia=RVgaia.append(nrow,ignore_index=True)
    k+=1
    if k>Ntot:break

In [11]:
cond=GAIA["hip"]!=''
sub1=GAIA[cond]
print("GAIA Hip not null:",len(sub1))
cond=RV["hip"]!=''
sub2=RV[cond]
print("RV Hip not null:",len(sub2))

sub1=sub1.head(1000)
sub2=sub2.head(1000)

GAIA Hip not null: 93635
RV Hip not null: 36820


In [12]:
#Merge
part1=pd.merge(left=sub1,right=sub2,left_on='hip',right_on='hip')
print("Number of matchings:",len(part1))

Number of matchings: 14


,hip,tycho2_id_x,solution_id,source_id,random_index,ref_epoch,ra,ra_error,dec,dec_error,...,l,b,ecl_lon,ecl_lat,tycho2_id_y,RAJ2000,DEJ2000,RV,eRV,CAT
0,13976,,1635378410781933568,8479090076267264,1206856,2015.0,45.013096,0.255069,7.749843,0.150187,...,169.311160,-43.244635,44.841330,-8.919388,,45.011721,7.749752,28.78,0.08,Maldonado2010.tsv
1,12505,,1635378410781933568,19873363434704512,663804,2015.0,40.226707,0.203716,8.240286,0.158182,...,163.663936,-45.796281,40.414952,-7.050246,,40.226700,8.240300,-9.20,0.10,GCS2011.tsv
2,11106,,1635378410781933568,22910936104830592,576407,2015.0,35.736131,0.257881,8.545439,0.218343,...,157.997061,-47.998351,36.270885,-5.359914,,35.735800,8.545300,2.00,0.40,GCS2011.tsv
3,12929,,1635378410781933568,25488741116165120,1180473,2015.0,41.573110,0.255083,11.774358,0.193228,...,162.306123,-42.150256,42.757742,-4.081856,,41.571991,11.775240,10.97,0.12,Maldonado2010.tsv
4,13223,,1635378410781933568,31986236361233920,1001992,2015.0,42.554342,0.228468,13.710432,0.151265,...,161.863987,-40.016425,44.247152,-2.515523,,42.553700,13.710300,-9.20,0.40,GCS2011.tsv
5,13122,,1635378410781933568,33216143195901312,1410431,2015.0,42.195270,0.218954,14.613597,0.252456,...,160.822170,-39.473052,44.179917,-1.549912,,42.195000,14.613600,30.10,0.40,GCS2011.tsv
6,13258,,1635378410781933568,33439206617713792,553947,2015.0,42.655207,0.254924,15.708267,0.173628,...,160.499449,-38.318804,44.925888,-0.633883,,42.653718,15.709914,-28.90,0.11,Maldonado2010.tsv
7,17496,,1635378410781933568,36590647460764288,75350,2015.0,56.214360,0.253371,11.920536,0.134403,...,175.819783,-32.614451,56.698553,-7.698612,,56.213026,11.920000,83.92,0.11,Maldonado2010.tsv
8,19076,,1635378410781933568,53112939972051968,941553,2015.0,61.335183,0.576866,22.008361,0.155267,...,171.447077,-22.094547,63.588142,1.158997,,61.334409,22.008905,24.16,0.09,Maldonado2010.tsv
9,17026,,1635378410781933568,64182342004153216,646725,2015.0,54.750259,0.596277,20.915618,0.379496,...,167.427495,-27.134808,57.366149,1.380780,,54.750000,20.916700,-7.60,1.00,Web1995-HIP.csv


In [ ]:
cond=GAIA["tycho2_id"]!=''
sub1=GAIA[cond]
print("GAIA Tycho2 not null:",len(sub1))
cond=RV["tycho2_id"]!=''
sub2=RV[cond]
print("RV Tycho2 not null:",len(sub2))

In [ ]:
part2=pd.merge(left=sub1,right=sub2,left_on='tycho2_id',right_on='TYC2')
print("Number of matchings:",len(part2))

In [ ]:
print(len(RV))

In [ ]:
print(len(GAIA))

In [ ]:
GAIA["hip"]

In [ ]:
len(GAIA[GAIA["tycho2_id"]!=''])

In [ ]:
len(RV[RV["TYC2"]!=''])

In [ ]:
len(RV[RV["HIP"]!=''])

In [14]:
RVGaia=pd.read_csv("../RVGaia/RVGaia.csv")

/Users/jzuluaga/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [15]:
RVGaia

,hip,tycho2_id,solution_id,source_id,random_index,ref_epoch,ra,ra_error,dec,dec_error,...,phot_variable_flag,l,b,ecl_lon,ecl_lat,RAJ2000,DEJ2000,RV,eRV,CAT
0,NaN,55-72-1,1635378410781933568,16870631694208,1081909,2015.0,45.112779,0.206981,0.380844,0.150943,...,NOT_AVAILABLE,176.665032,-48.556851,42.764140,-16.004886,45.11283,0.38092,2.061,1.073,RAVE-DR5.tsv
1,NaN,48-1138-1,1635378410781933568,86105504493056,1911572,2015.0,44.797149,0.794706,0.630677,0.485048,...,NOT_AVAILABLE,176.061442,-48.607747,42.526411,-15.673636,44.79708,0.63086,-32.497,0.844,RAVE-DR5.tsv
2,NaN,55-1269-1,1635378410781933568,180045029052928,15459,2015.0,46.123388,0.151871,1.066354,0.532063,...,NOT_AVAILABLE,176.977602,-47.353826,43.975631,-15.640883,46.12342,1.06636,-5.475,1.515,RAVE-DR5.tsv
3,NaN,55-1181-1,1635378410781933568,202619377158912,30919,2015.0,46.044931,0.212695,1.230477,0.629442,...,NOT_AVAILABLE,176.723034,-47.296261,43.946408,-15.461170,46.04492,1.23061,-10.742,1.079,RAVE-DR5.tsv
4,NaN,55-1308-1,1635378410781933568,262439682250240,1226965,2015.0,45.939012,0.200155,1.567219,0.194502,...,NOT_AVAILABLE,176.259169,-47.137079,43.941250,-15.108207,45.93912,1.56728,-13.173,1.558,RAVE-DR5.tsv
5,NaN,48-685-1,1635378410781933568,310337157179392,1978867,2015.0,44.356001,0.182949,0.998117,0.134511,...,NOT_AVAILABLE,175.196419,-48.660561,42.201067,-15.192770,44.35617,0.99831,-3.081,1.099,RAVE-DR5.tsv
6,NaN,48-120-1,1635378410781933568,383111082972928,1727679,2015.0,43.837509,0.285042,1.323709,0.217455,...,NOT_AVAILABLE,174.289624,-48.790621,41.788140,-14.728455,43.83746,1.32386,-34.862,2.549,RAVE-DR5.tsv
7,NaN,48-1076-1,1635378410781933568,407781375177472,247358,2015.0,44.263441,0.268377,1.501589,0.191004,...,NOT_AVAILABLE,174.564023,-48.366366,42.263052,-14.684380,44.26333,1.50153,55.321,0.750,RAVE-DR5.tsv
8,NaN,48-1215-1,1635378410781933568,450559249338752,2014341,2015.0,44.993785,0.210021,1.598919,0.154719,...,NOT_AVAILABLE,175.240816,-47.784978,43.014382,-14.805026,44.99379,1.59911,-27.494,1.722,RAVE-DR5.tsv
9,NaN,55-551-1,1635378410781933568,461142048768128,936913,2015.0,45.410808,0.452906,1.656348,0.343151,...,NOT_AVAILABLE,175.618395,-47.449886,43.444318,-14.870929,45.41083,1.65639,17.492,1.130,RAVE-DR5.tsv


In [16]:
RVGaia=RVGaia.fillna('NULL')

TypeError: Could not compare [''] with block values